<a href="https://colab.research.google.com/github/usshaa/SMBDA/blob/main/C-3.2%3A%20MapReduce_Handson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**MapReduce (Basic Job Execution)**
###What We will Learn in this :
####how to run a MapReduce job, including creating and executing jobs on HDFS.




# Install Java Development Kit (JDK

In [1]:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# setup Hadoop as told in previous class

In [2]:
!tar -xzf /content/drive/MyDrive/hadoop-setup/hadoop-3.3.6.tar.gz -C /content/

tar (child): /content/drive/MyDrive/hadoop-setup/hadoop-3.3.6.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


# setting up environment

In [3]:
# Set Hadoop environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
os.environ["PATH"] = os.environ["PATH"] + ":" + os.environ["HADOOP_HOME"] + "/bin" + ":" + os.environ["HADOOP_HOME"] + "/sbin"

# Configure Hadoop (replace with your desired configuration)
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64" >> ~/.bashrc
!echo "export HADOOP_HOME=/content/hadoop-3.3.6" >> ~/.bashrc
!echo "export PATH=$PATH:$HADOOP_HOME/bin:$HADOOP_HOME/sbin" >> ~/.bashrc

# Source the bashrc file to apply the changes
!source ~/.bashrc

# Verify Hadoop installation
!hadoop version

/bin/bash: line 1: hadoop: command not found


# verfying the working of haddop

In [4]:
!jps

546 Jps


# so initially will create an input.txt file

In [5]:

with open('/content/new_folder/input.txt', 'w') as f:
    f.write("Hello HadoopHadoop MapReduce is great for big dataBig data can be processed using Hadoop MapReduce is a programming model")

FileNotFoundError: [Errno 2] No such file or directory: '/content/new_folder/input.txt'

In [ ]:
!hadoop jar /content/hadoop-3.3.6/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.6.jar wordcount /content/new_folder/input.txt /content/new_folder/output

In [ ]:
#displaying the result of MapReduce WordCount Program
!hdfs dfs -cat /content/new_folder/output/part-r-00000

**So here we have seen excution of MapReduce using Existing jar Files**

# Working On New dataset using own Scripting Of map Reduce

**Case Study 2: Titanic Dataset - Survival Analysis**
**Dataset Source:**
Titanic Dataset

**Task:**
Count the number of survivors and non-survivors.

**Steps:**
Prepare the Data: Download and upload



1. the train.csv file to /content/newfolder.

2. Write the MRJob Script: Save this as survival_analysis.py.

### downloading the dataset

In [12]:
#!/bin/bash
!kaggle datasets download hesh97/titanicdataset-traincsv

Dataset URL: https://www.kaggle.com/datasets/hesh97/titanicdataset-traincsv
License(s): CC0-1.0
  0% 0.00/22.0k [00:00<?, ?B/s]
100% 22.0k/22.0k [00:00<00:00, 36.1MB/s]


In [14]:
!unzip titanicdataset-traincsv.zip -d /content/new_folder

Archive:  titanicdataset-traincsv.zip
  inflating: /content/new_folder/train.csv  


### viewing the dataset

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


df= pd.read_csv('/content/new_folder/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### writing the script to count the passenger

In [18]:
!pip install mrjob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 5.7 MB/s eta 0:00:00


In [20]:
%%writefile survival_analysis.py
from mrjob.job import MRJob

class SurvivalAnalysis(MRJob):

    def mapper(self, _, line):
        fields = line.split(',')
        if fields[0] != "PassengerId":  # Skip the header
            survived = fields[1]
            yield survived, 1

    def reducer(self, survived, counts):
        yield survived, sum(counts)

if __name__ == "__main__":
    SurvivalAnalysis.run()


Overwriting survival_analysis.py


# Run the Script:

In [21]:
!python3 survival_analysis.py /content/new_folder/train.csv > /content/new_folder/survival_analysis_output.txt


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/survival_analysis.root.20250211.100104.434779
Running step 1 of 1...
job output is in /tmp/survival_analysis.root.20250211.100104.434779/output
Streaming final output from /tmp/survival_analysis.root.20250211.100104.434779/output...
Removing temp directory /tmp/survival_analysis.root.20250211.100104.434779...


#View the Results:

In [22]:
!cat /content/new_folder/survival_analysis_output.txt


"1"	342
"0"	549


here we have found that 342 paasenger survived where as 549 not